# 卫星数据集预处理

## 数据说明及描述

本课程提供了脱敏后的卫星传感器采集到的数据，用来进行数据分析以及健康监测。本节课我们使用三个分系统的数据：
1. 激光载荷
2. 供配电 
3. 姿轨控

每个数据集都包含故障数据和正常数据，以`csv`格式保存在相应的子文件夹中。结构如下。

```
train/
    ├── 激光载荷/
    │   ├── 故障数据.csv
    │   └── 正常数据.csv
    ├── 供配电/
    │   ├── 故障数据.csv
    │   └── 正常数据.csv
    └── 姿轨控/
        ├── 故障数据.csv
        └── 正常数据.csv
```

## 数据清洗
数据清洗是指识别并纠正或删除数据集中错误、重复或不完整的数据，以提高数据质量和分析准确性。一般数据清洗包含如下步骤：

1. 数据审查：
- 检查数据的完整性和一致性。
- 初步识别缺失值、重复值和异常值。
2. 处理缺失值：
- 删除包含缺失值的记录。
- 用平均值、中位数或其他统计方法填补缺失值。
- 使用插值或机器学习模型预测缺失值。
3. 去除重复值：
- 识别并删除重复记录。
4. 处理异常值：
- 识别异常值，并决定是否删除或更正它们。
5. 标准化数据：
- 统一数据格式（如日期格式）。
- 统一单位（如将不同单位的值转换为统一单位）。
6. 数据转换：
- 根据需要进行数据类型转换（如字符串转换为数值）。
- 提取或衍生新特征。
7. 一致性检查：
- 确保数据在不同数据源之间的一致性。
8. 验证和报告：
- 验证数据清洗效果。
- 记录数据清洗过程，以便追溯和审核。

本课程我们主要使用numpy和pandas库进行数据清洗。numpy是python中最基础的数值计算库，文档官网为`https://numpy.org/doc/stable/index.html`。pandas库是python中最常用的数据处理库，它提供了丰富的数据处理函数，文档官网为`https://pandas.pydata.org/docs/`

In [11]:
# 导入必要的包
import numpy as np
import pandas as pd
import json
import os

In [12]:
def load(data_paths):
    '''处理标签并合并正常和故障数据集'''
    datasets = {}
    for name, path in data_paths.items():
        fault_path = os.path.join(path, "故障数据.csv")
        normal_path = os.path.join(path, "正常数据.csv")
        df_fault = pd.read_csv(fault_path, header=0)
        df_normal = pd.read_csv(normal_path, header=0)

        df_normal['label'] = 0  # 正常数据标签为0

        labels = df_fault['label']

        # 生成编码字典
        atoi = {"正常": 0}
        current_code = 1  # 从 1 开始编码
        for cls in labels.unique():
            atoi[cls] = current_code
            current_code += 1  # 更新编码

        # 生成逆向映射字典
        itoa = {i: cls for cls, i in atoi.items()}

        save_path = path
        with open(os.path.join(save_path, 'atoi.json'), 'w', encoding='utf-8') as f:
            json.dump(atoi, f, ensure_ascii=False)

        with open(os.path.join(save_path, 'itoa.json'), 'w', encoding='utf-8') as f:
            json.dump(itoa, f, ensure_ascii=False)
        print("编码字典已保存：")
        print(atoi)

        # 应用编码
        df_fault['label'] = df_fault['label'].map(atoi)

        # 合并数据
        df = pd.concat([df_fault, df_normal], ignore_index=True)
        print(f"合并后数据形状: {df.shape}")
        datasets[name] = df

    return datasets


In [13]:
dataset_paths = {
    '激光载荷': '../../data/train/激光载荷',
    '供配电': '../../data/train/供配电', 
    '姿轨控': '../../data/train/姿轨控'
}
datasets = load(dataset_paths)

/tmp/ipykernel_74930/450373922.py:8: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_normal = pd.read_csv(normal_path, header=0)


编码字典已保存：
{'正常': 0, '俯仰轴解锁异常': 1, '转台俯仰轴精度下降': 2, '转台方位轴精度下降': 3, '章动跟踪快反镜执行体故障': 4, '转台俯仰轴电机故障': 5, '转台俯仰轴码盘故障': 6, '转台俯仰轴驱动器故障': 7, '转台方位轴电机故障': 8, '转台方位轴码盘故障': 9, '转台方位轴驱动器故障': 10, 'EDFA光输出中断': 11, '信号激光器故障': 12, 'SWIR图像探测器有坏点': 13, '精跟踪快反镜1执行体故障': 14}
合并后数据形状: (42964, 133)
编码字典已保存：
{'正常': 0, '下位机通信异常或死机': 1, '五串电池开路故障': 2, '五串电池性能严重衰降': 3, '五串电池片功率损失故障': 4, '五串电池短路故障': 5, '单串电池片开路': 6, '单串电池片短路': 7, '单体电池开路': 8, '单体电池短路': 9, '第13级S3R模块只能供电故障': 10, '第13级S3R模块只能分流故障': 11, '第17级S3R模块只能供电故障': 12, '第17级S3R模块只能分流故障': 13, '蓄电池组加热带误断': 14, '蓄电池组加热带误通': 15}
合并后数据形状: (22784, 56)
编码字典已保存：
{'正常': 0, '磁力矩器X输出磁矩下降故障': 1, '太阳敏感器1故障': 2, '负Y翼A轴故障': 3, '陀螺2输出常零故障': 4, '陀螺1输出常值故障': 5, '动量轮1输出力矩下降故障': 6, '星敏感器1输出噪声变大故障': 7, '动量轮2输出力矩下降故障': 8, '动量轮3输出力矩下降故障': 9, '动量轮4输出力矩下降故障': 10}
合并后数据形状: (40329, 96)


In [14]:
# 查看处理前的信息, 以确定无关变量
for name, df in datasets.items():
    print()
    print(f"数据集：{name}")
    print(df.shape)

    print("前五行信息：")
    print(df.head())

    print("单一值：")
    constant_cols = [
        col for col in df.columns
        if df[col].nunique() <= 1
    ]
    print(constant_cols)

    print("各列信息：")
    for col in df.columns:
        print(f"列名: {col}, 数据类型: {df[col].dtype}, 缺失值数量: {df[col].isnull().sum()}")
        print("唯一值及其计数：")
        print(df[col].value_counts(dropna=False).head(10))  # 显示前10个唯一值及其计数
        print()

    print("全部信息")
    print(df.info())


数据集：激光载荷
(42964, 133)
前五行信息：
   卫星时间秒 终端紧急关机 激光处理机工作模式 工作阶段  开机计时  广播帧接收计数 姿态测量时间整数秒  姿态测量时间秒小数   本地姿态X姿态角  \
0  T+20s     正常      入轨模式   待机    20       10     T+20s      0.851  25.001074   
1  T+22s     正常      入轨模式   待机    22       12     T+22s      0.850  25.001173   
2  T+24s     正常      入轨模式   待机    24       14     T+24s      0.849  25.001003   
3  T+26s     正常      入轨模式   待机    26       16     T+26s      0.849  25.001190   
4  T+28s     正常      入轨模式   待机    28       18     T+28s      0.853  25.001195   

   本地姿态Y姿态角  ...  EDFA主放（备）二级泵浦电流  EDFA主放（主）中间级光功率  EDFA主放（备）中间级光功率  \
0  0.133734  ...                0              0.0                0   
1  0.133836  ...                0              0.0                0   
2  0.133849  ...                0              0.0                0   
3  0.133942  ...                0              0.0                0   
4  0.133969  ...                0              0.0                0   

   EDFA主放（主）故障状态自检反馈  EDFA主放（备）故障状态自检反馈  SWIR_CCD最大灰度值  

In [20]:
def preprocess(datasets):
    """
    对三个数据集分别进行预处理;
    去除空值，去除单一值，去除无关变量，处理名义变量
    """
    processed_datasets = {}
    for name, df in datasets.items():
        df = df.copy()
        print(f"正在处理数据集: {name}")

        # 去除单一值, 数值和名义变量均处理
        df = df.loc[:, df.nunique() > 1]

        # 手动处理时间等无关特征
        if name == "供配电":
            # 按时间排序并删除掉时间特征
            # 处理时间字符串，将时间转换为时间戳
            df['时间'] = df['时间'].apply(lambda x: pd.to_datetime(x).timestamp())
            df = df.sort_values(by='时间')
            df.drop(['时间'], axis=1, inplace=True)
        
        elif name == "激光载荷":
            df.drop(['开机计时', '姿态测量时间整数秒', '姿态测量时间秒小数', 
                     '广播帧接收计数', '激光处理机接收正确帧计数'], axis=1, inplace=True)
            # 处理卫星时间秒变量，对不同格式的时间进行处理
            df['卫星时间秒'] = df['卫星时间秒'].apply(lambda x: float(x) if isinstance(x, int)
                                            else float(x.split('+')[1][:-1]) if x.startswith('T+')
                                            else -float(x.split('-')[1][:-1]) if x.startswith('T-')
                                            else float(x))
            # 按卫星时间秒排序
            df = df.sort_values(by='卫星时间秒')
            df.drop(['卫星时间秒'], axis=1, inplace=True)

        # elif name == "姿轨控":
            # df.drop([''], axis=1, inplace=True)

        # 处理缺失值（实际上只有姿轨控有缺失值）
        for col in df.columns:
            if df[col].isnull().sum() > 0:
                if df[col].dtype == 'object':
                    # 名义变量用众数填充
                    mode_value = df[col].mode()[0]
                    df[col].fillna(mode_value, inplace=True)
                else:
                    # 数值变量用均值填充
                    mean_value = df[col].mean()
                    df[col].fillna(mean_value, inplace=True)
        
        object_col = df.select_dtypes(include=['object']).columns
        # 名义变量转为枚举映射
        object_maps = {} #保存map保证后续使用时统一映射
        for col in object_col:
            unique_mode = sorted(df[col].unique()) # 排序保证映射一致
            mode_map = {mode: idx for idx, mode in enumerate(unique_mode)}
            df[col] = df[col].map(mode_map) # 直接改df
            object_maps[col] = mode_map
        save_path = os.path.join(dataset_paths[name], 'object_to_enum.json')
        print(f"名义变量映射字典：{object_maps}")
        with open(save_path, 'w', encoding='utf-8') as f:
            json.dump(object_maps, f, ensure_ascii=False)
        # 反向映射保存
        enmu_to_object_maps = {}
        for col, mapping in object_maps.items():
            enmu_to_object_map = {v: k for k, v in mapping.items()}
            enmu_to_object_maps[col] = enmu_to_object_map
        save_path = os.path.join(dataset_paths[name], 'enum_to_object.json')
        with open(save_path, 'w', encoding='utf-8') as f:
            json.dump(enmu_to_object_maps, f, ensure_ascii=False)
            
        processed_datasets[name] = df
        
    return processed_datasets

In [21]:
processed_datasets = preprocess(datasets)

正在处理数据集: 激光载荷
名义变量映射字典：{'激光处理机工作模式': {'休眠模式': 0, '入轨模式': 1, '待机模式': 2, '通信模式': 3}, '工作阶段': {'关机': 0, '关机准备阶段': 1, '待机': 2, '捕获阶段/解锁完成': 3, '瞄准阶段/解锁中': 4, '章动阶段': 5, '粗跟踪阶段': 6, '精跟踪阶段': 7, '通信阶段': 8}, '激光处理机最后一条接收指令编号': {'0x01': 0, '0x05': 1, '0x0F': 2, '0x19': 3, '0x1D': 4}, '模块工作状态-EDFA': {'已加电': 0, '断电': 1}, '模块工作状态-可见光图像传感器': {'已加电': 0, '断电': 1}, '模块工作状态-SWIR图像传感器': {'已加电': 0, '断电': 1}, '模块工作状态-快反镜': {'已加电': 0, '断电': 1}, '模块工作状态-转台': {'已加电': 0, '断电': 1}, '模块工作状态-信号激光器': {'已加电': 0, '断电': 1}, '模块工作状态-数据板': {'已加电': 0, '断电': 1}, '模块工作状态-PAT板': {'已加电': 0, '断电': 1}, '转台方位轴控温点1': {'已加电': 0, '断电': 1}, '转台方位轴控温点2': {'已加电': 0, '断电': 1}, '转台高度轴控温点1': {'已加电': 0, '断电': 1}, '转台高度轴控温点2': {'已加电': 0, '断电': 1}, '光学基板控温点1': {'已加电': 0, '断电': 1}, '光学基板控温点2': {'已加电': 0, '断电': 1}, '光学基板控温点3': {'已加电': 0, '断电': 1}, '模块工作状态-锁紧装置': {'装置已解锁': 0, '装置锁定': 1}, '开环扫描状态': {'完成扫描': 0, '未扫描': 1, '正在扫描': 2}, '章动跟踪状态': {'未章动跟踪': 0, '正在章动扫描': 1, '进入章动跟踪': 2}, 'EDFA主放（主）故障状态自检反馈': {'正常': 0, '输出故障': 1}}
正在处理数据集: 供配电
名义变量

/tmp/ipykernel_74930/3966286664.py:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mean_value, inplace=True)
/tmp/ipykernel_74930/3966286664.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

名义变量映射字典：{'工作模式': {'主动段模式(0)': 0, '偏置飞行模式(5)': 1, '入轨段控制模式(1)': 2, '全姿态捕获(8)': 3, '姿态无控模式(9)': 4, '姿态机动模式(4)': 5, '正常对地运行模式(2)': 6, '轨控模式(7)': 7}, '子模式_全捕模式': {'太阳搜索子模式(1)': 0, '对日定向子模式(2)': 1, '星敏全天球搜索(7)': 2, '星敏地球捕获子模式(8)': 3, '速率阻尼子模式(0)': 4}, '子模式_入轨段控制模式': {'分离角速度判断(1)': 0, '帆板展开(3)': 1, '速率阻尼(2)': 2}, '模拟太阳1a角见太阳标志': {'未见太阳(0)': 0, '见太阳(1)': 1}, '模拟太阳1B角见太阳标志': {'未见太阳(0)': 0, '见太阳(1)': 1}, '模拟太阳2a角见太阳标志': {'未见太阳(0)': 0, '见太阳(1)': 1}, '模拟太阳2B角见太阳标志': {'未见太阳(0)': 0, '见太阳(1)': 1}, '磁力矩器Z输出磁矩方向': {'正磁矩(0)': 0, '负磁矩(1)': 1}, '磁力矩器Y2输出磁矩方向': {'正磁矩(0)': 0, '负磁矩(1)': 1}, '磁力矩器Y1输出磁矩方向': {'正磁矩(0)': 0, '负磁矩(1)': 1}, '磁力矩器X输出磁矩方向': {'正磁矩(0)': 0, '负磁矩(1)': 1}, '正Y翼B轴控制模式': {'反向转动模式(2)': 0, '归零模式(4)': 1, '待命模式(0)': 2, '正向转动模式(1)': 3}, '正Y翼A轴控制模式': {'保持模式(3)': 0, '反向转动模式(2)': 1, '归零模式(4)': 2, '待命模式(0)': 3, '正向转动模式(1)': 4}, '负Y翼B轴控制模式': {'反向转动模式(2)': 0, '归零模式(4)': 1, '待命模式(0)': 2, '正向转动模式(1)': 3}, '负Y翼A轴控制模式': {'保持模式(3)': 0, '反向转动模式(2)': 1, '归零模式(4)': 2, '待命模式(0)': 3, '正向转动模式(1)': 4}, '双轴太阳1滚动

In [17]:
# 查看处理后信息：
print("\n处理后数据集信息：")
for name, df in processed_datasets.items():
    print()
    print(f"数据集：{name}")
    print(df.shape)

    print("前五行信息：")
    print(df.head())

    print("描述性统计信息：")
    print(df.describe())

    # print("各列信息：")
    # for col in df.columns:
    #     print(f"列名: {col}, 数据类型: {df[col].dtype}, 缺失值数量: {df[col].isnull().sum()}")
    #     print("唯一值及其计数：")
    #     print(df[col].value_counts(dropna=False).head(10))  # 显示前10个唯一值及其计数
    #     print()

    # print("全部信息")
    # print(df.info())


处理后数据集信息：

数据集：激光载荷
(42964, 94)
前五行信息：
       激光处理机工作模式  工作阶段   本地姿态X姿态角  本地姿态Y姿态角  本地姿态Z姿态角  本地姿态X姿态角速度  本地姿态Y姿态角速度  \
42954          0     0  25.000715  0.129329 -0.059939   -0.000008   -0.000113   
35818          0     0  25.000715  0.129329 -0.059939   -0.000008   -0.000113   
35633          0     0  25.000715  0.129329 -0.059939   -0.000008   -0.000113   
40408          0     0  25.000715  0.129329 -0.059939   -0.000008   -0.000113   
30608          0     0  25.000715  0.129329 -0.059939   -0.000008   -0.000113   

       本地姿态Z姿态角速度  本地姿态定姿四元数矢部q1  本地姿态定姿四元数矢部q2  ...  跟踪1S内最大光功率值  \
42954    0.000228       0.806301      -0.422884  ...          0.0   
35818    0.000228       0.806301      -0.422884  ...          0.0   
35633    0.000228       0.806301      -0.422884  ...          0.0   
40408    0.000228       0.806301      -0.422884  ...          0.0   
30608    0.000228       0.806301      -0.422884  ...          0.0   

       跟踪1S内最小光功率值  跟踪1S内平均光功率值  EDFA主放（主）一级泵浦电流  EDFA主放（主

In [18]:
# 为每个数据集创建训练集和测试集
def create_train_test_split(df, dataset_name):
    """
        对数据集划分训练集和测试集，保证方法可行性。
        确保每类故障都有代表性样本
    """
    print(f"\n=== 为 {dataset_name} 创建训练集和测试集 ===")
    
    test_indices = []
    for label_value in df['label'].unique():
        label_indices = df[df['label'] == label_value].index
        n = len(label_indices)
        test_count = max(1, n // 3)  # 至少保证每类有1个测试样本
        
        # 均匀采样 - 每3个取1个，保证取到约1/3
        if test_count > 0 and n >= 3:
            sampled = label_indices[::max(1, n // test_count)]
        else:
            # 如果样本太少，随机取一个（如果有多个的话）
            sampled = label_indices[:1] if len(label_indices) > 0 else []
        
        test_indices.extend(sampled)
        print(f"标签 {label_value}: 总样本 {n}, 测试样本 {len(sampled)}")
    
    test = df.loc[test_indices]
    train = df.drop(test_indices)
    
    print(f"训练集形状: {train.shape}")
    print(f"测试集形状: {test.shape}")
    print(f"训练集标签分布: {dict(train['label'].value_counts().sort_index())}")
    print(f"测试集标签分布: {dict(test['label'].value_counts().sort_index())}")
    
    return train, test

In [19]:
# 保存每个数据集的结果
for name, df in processed_datasets.items():
    df = df.copy()

    # 创建训练集和测试集
    train_df, test_df = create_train_test_split(df, name)
    
    # 保存数据集
    save_path = dataset_paths[name]
    all_path = os.path.join(save_path, f"processed_all.csv")
    train_path = os.path.join(save_path, f"processed_train.csv")
    test_path = os.path.join(save_path, f"processed_test.csv")
    
    df.to_csv(all_path, index=False)
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)
    
    print(f"保存 {name} 数据集:")
    print(f"- 完整数据: {all_path}")
    print(f"- 训练集: {train_path}")
    print(f"- 测试集: {test_path}")

print(f"\n所有数据集处理完成！")


=== 为 激光载荷 创建训练集和测试集 ===
标签 0: 总样本 37864, 测试样本 12622
标签 14: 总样本 68, 测试样本 23
标签 12: 总样本 18, 测试样本 6
标签 1: 总样本 240, 测试样本 80
标签 13: 总样本 48, 测试样本 16
标签 11: 总样本 34, 测试样本 12
标签 2: 总样本 2018, 测试样本 673
标签 3: 总样本 2018, 测试样本 673
标签 4: 总样本 620, 测试样本 207
标签 6: 总样本 6, 测试样本 2
标签 5: 总样本 6, 测试样本 2
标签 10: 总样本 6, 测试样本 2
标签 9: 总样本 6, 测试样本 2
标签 7: 总样本 6, 测试样本 2
标签 8: 总样本 6, 测试样本 2
训练集形状: (28640, 94)
测试集形状: (14324, 94)
训练集标签分布: {0: np.int64(25242), 1: np.int64(160), 2: np.int64(1345), 3: np.int64(1345), 4: np.int64(413), 5: np.int64(4), 6: np.int64(4), 7: np.int64(4), 8: np.int64(4), 9: np.int64(4), 10: np.int64(4), 11: np.int64(22), 12: np.int64(12), 13: np.int64(32), 14: np.int64(45)}
测试集标签分布: {0: np.int64(12622), 1: np.int64(80), 2: np.int64(673), 3: np.int64(673), 4: np.int64(207), 5: np.int64(2), 6: np.int64(2), 7: np.int64(2), 8: np.int64(2), 9: np.int64(2), 10: np.int64(2), 11: np.int64(12), 12: np.int64(6), 13: np.int64(16), 14: np.int64(23)}
保存 激光载荷 数据集:
- 完整数据: ../../data/train/激光载荷/processed_all.